In [ ]:
import cv2
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans

def display_img(img, fgsize=(5,5), **kwargs):
    
    plt.figure(figsize=fgsize)
    plt.imshow(img, **kwargs)
    plt.show()


# 10.1 Cameraman
## 10.1.1
Laden Sie das Bild 'cam.tif'. Verwenden sie dafür die Funktion *imread* von OpenCV. Lassen sie sich das gelesene Bild mittels der Funktion *display_img* anzeigen.

# 10.1.2
Eine Rauschreduktion im Ortsraum wird im Folgenden auf zwei unterschiedliche Weisen durchgeführt: Mittelwertbildung und Medianfilter.

Wenden Sie einen Gaußschen Weichzeichner (gaussian blur) auf das zuvor geladene Bild an. 
Verwenden sie dafür die Funktion *GaussianBlur* von OpenCV. 
Zeigen sie das gefilterte Bild an.
Welchen Einfluss haben die unterschiedlichen Parameter?

Wenden Sie einen Medianfilter auf das zuvor geladene Bild an. Verwenden sie dafür die Funktion *medianBlur* von OpenCV. Zeigen sie das gefilterte Bild an.

# 10.2 Büroklammern

## 10.2.1
Laden Sie das Bild clip.tif.
Konvertieren Sie das geladene Bild in den Datentyp "8bit unsigned int" mittels der numpy Funktion *np.astype*.
Zeigen Sie das Ergebnis an.

Filtern Sie das Bild mittels einer Schwellwertstrategie. 
Sie können hierfür die Funktion *cv2.threshold* verwenden.
Probieren Sie verschiedene Schwellwerte aus.
Was beobachten Sie?

## 10.2.2
Trennen Sie das Nutz- und Störsignal im Ortsraum vor der Anwendung des Schwellwertes.

# 10.3 Satellitenbilder

Im folgenden wird eine Landaufnahme des Sentinel-2 Satelliten betrachtet. 
Die betrachtete Aufnahme beinhaltet nur eine Auswahl an 10 gemessenen Wellenlängenbereichen.
Jedes Satellitenband soll in einem Kanal des Bildes img_mspec platziert werden.

| Sentinel-2 Bänder   |  Wellenlänge [micrometer]  | Kanal |
|:----------|-------------| --- |
| Band 2 |  0.490 | 0 |
| Band 3 |  0.560 | 1 |
| Band 4 |  0.665 | 2 |
| Band 5 |  0.705 | 3 |
| Band 6 |  0.740 | 4 |
| Band 7 |  0.783 | 5 |
| Band 8 |  0.842 | 6 |
| Band 8A |  0.865 | 7 |
| Band 11 |  1.610 | 8 |
| Band 12 |  2.190 | 9 |

In [ ]:
# wir laden das multispektrale Satellitenbild
path = r'sentinel2_sample'
bname = 'RT_S2A_OPER_MSI_L1C_TL_SGS__20160506T153005_A004552_T32TQM_B'
chan_names = ('02', '03', '04', '05', '06', '07', '08', '8A', '11', '12')
img_mspec = np.asarray([np.array(Image.open(os.path.join(path, bname+chan_name+'.tif'))) for chan_name in chan_names])
img_mspec = img_mspec.transpose((1,2,0))

# betrachte nur einen kleinen Bildauschnitt für eine schnellere Verarbeitungsgeschwindigkeit
npix = 500
img_mspec = img_mspec[:npix, :npix, :]

# ziehe nur herkömmliche "RGB" Daten für Visusalisierungszwecke heran
img_rgb = img_mspec[:,:,(2,1,0)] # die Kanäle 0,1,2 entsprechen B,G,R (im weitesten Sinne)
img_rgb = (img_rgb*255).astype(np.uint8)

display_img(img_rgb, fgsize=(15,15))

In [ ]:
# Wie zu sehen ist, sind rohe Kamera RGB Werte nicht für eine direkte menschliche Betrachtung geeignet. Eine entsprechende
# Umrechnung sprengt den Rahmen dieses Versuchs. Deshalb bedienen wir uns einem naiven Trick zu besseren Visualisierung:
# Kontrastverstärkung durch Manipulation der "Helligkeit"
img_lab  = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
img_lab[:,:,0] = 2 * img_lab[:,:,0]
img_output = cv2.cvtColor(img_lab, cv2.COLOR_LAB2RGB)
display_img(img_output, fgsize=(15,15))

Eine einfache Art multispektrale Satellitenbilder zu visualisieren ist es drei individuelle Bänder jeweils als eine der drei Grundfarben RGB zu interpretieren.
Ein so entstandendes Bild wird als Farbkomposit (color composite) bezeichnet.
Ein Farbkomposit wird durch die Schreibweise: "RGB = Br Bg Bb" gekennzeichnet, wobei Br, Bg und Bb jeweils die Bänder beschreiben, die mit dem entsprechenden Farbkanal assoziiert sind.

## 10.3.1
Visualisieren Sie den Farbkomposit RGB = (8 8A 12)    

Das Satellitenbild soll nun automatisch klassifiziert werden. Dafür wird ein Verfahren aus dem Bereich "Unsupervised Learning" eingesetzt: KMeans-Clustering

## 10.3.2
Klassifizierten Sie das multispektrale Satellitenbild mittels des KMeans-Algorithmus. 
Verwenden Sie initial eine Clusteranzahl von drei. 
Visualisieren Sie das Ergebnis. Welche drei Klassen sind gelernt worden?

# 10.4 Dota
Eine periodische Störung soll aus einem Bild entfernt werden. Dafür wird zuerst das Bild inklusive Störung mittels des folgenden Codes erzeugt.

## 10.4.1

In [ ]:
# Unser allzeit geliebtes Videospiellogo wird geladen. Es sei hervorgehoben, dass erstmalig ein herkömmliches Farbbild geladen 
# wird.
img_dota = cv2.imread('dota2.jpg')

# Fast alle Standardbibliotheken innerhalb von Python unterstützen nativ keine anderen Formate als "sRGB". Das kanonische 
# Format von OpenCV ist allerdings BGR. Um mögliche Verwirrungen zu vermeiden, ändern wir einfach die Kanalreihenfolge.
b,g,r = cv2.split(img_dota)       
img_dota = cv2.merge([r,g,b])

display_img(img_dota)

In [ ]:
# Eine periodische Störung wird simuliert. Hierbei handelt es sich um eine einfache Sinuswelle.
kx = 10
ky = 5
scale = np.sqrt(kx**2 + ky**2)
norm = np.array([kx, ky]) / scale

img_wave = np.zeros((img_dota.shape[0], img_dota.shape[1]))
for row in range(img_dota.shape[0]):
    for col in range(img_dota.shape[1]):
        val = np.array([row, col] @ norm) * 2 * math.pi / scale
        img_wave[row, col] = math.cos(val)

# Die Sinuswelle wird gleichermaßen in jedem Farbkanal wiederholt
img_wave = np.repeat(np.expand_dims(img_wave, 2), 3, axis = 2)

display_img(img_wave)

In [ ]:
# Das gestörte Gesamtbild ergibt sich aus der gewichteten Überlagerung von Originalbild und Störung
img = ((img_dota) + img_wave*30).astype(np.uint8)
display_img(img)

Der Einfachheit halber rechnen wir im Folgenden nur mit dem Grauwertbild weiter:

In [ ]:
img_dota_gray = cv2.cvtColor(img_dota, cv2.COLOR_RGB2GRAY)

# auch das Grauwertbild wird gestört
img = ((img_dota_gray) + img_wave[:,:,0]*30)

# Um das Bild sinnvoll darstellen zu können, muss es auf das Interval [0, 1] normalisiert werden. 
maxval = np.amin(img)
minval = np.amax(img)
img_norm = (img-minval) / (maxval-minval)

display_img(img, cmap='gray')

## 10.4.1
Berechnen und Visualisieren Sie das Amplitudenspektrum von img. Sie benötigen dafür die Funktionen *np.fft.fft2*, *np.fft.fftshift*, *np.log10* und *np.abs*.

Berechnen und Visualisieren Sie das Amplitudenspektrum von img_dota_gray.

# 10.4.2
Die Störung soll nun im Ortsfrequenzraum entfernt werden. Welche Art von Filter wird dafür benötigt? 

Im Folgenden ist eine Hilfsfunktion für die Erstellung einer geeigneten Filtermaske gegeben. Erstellen Sie damit die Filtermaske.

In [ ]:
def create_circular_mask(target_shape, radius):
    height = target_shape[0]
    width = target_shape[1]
    mask = np.zeros((height, width))
    for row in range(height):
        for col in range(width):
            if (row-height/2)**2 + (col-width/2)**2 <= radius**2:
                mask[row, col] = 1
    
    return mask

# 10.4.3
Filtern Sie das gestörte Logo im Ortsfrequenzraum. Wandeln das Ergebnis anschließend zurück in den Ortsraum. Sie benötigen an neuen Funktionen *np.fft.ifftshift* und *np.fft.ifft2*. Zeigen Sie das Ergebnis an.